# <font color='#0f2f5c'>The Linguistic Locator: The marriage between Natural Language Processing and paleography.</font>

### <font color='#0f2f5c'>Table of Contents</font>

* [Data Gathering](#DataPrep)
* [General use Methods](#GUM)
    * [Decorators](#GUM_Decorators)
    * [Methods](#GUM_Methods)
    * [Thread Manager](#GUM_ThreadManager)

### As we go further and further back in history, it gets harder and harder to pin down when and where a certain piece of writing comes from. Generally, writing gets dated by looking at the handwriting and general context of history the text fits in. A text could also be dated by looking at the material it was written on itself. These fields of analysis are called paleography and codicology respectively.
### This work still gets done by humans, but it might be possible to predict these attributes computationally. By pondering this, we arrive at the main question: What models of AI can accurately predict attributes of historical writings?

### In order to answer this question, the following sub questions should be answered:
1. How accurate does the model need to be?
2. What models could possibly work for this application?
3. What criteria do the writings need to meet?
4. What attributes can we predict (accurately)?
5. What model works best?

### To test this, some data should be gathered to train a potential model on. In order to do this, some criteria need to be set up and the scope of this project should be defined.
### To start off, let's only train on texts from 1200 until 1950. 1200 is an arbitrary pick. Going earlier would get progressively harder and harder to find proper examples for that have been dated already. 1900 works as a proper endpoint, because going any more recent slowly moves us from historical to contemporary writings.

### The first version of this project will only train on the following languages:
1. Dutch
2. German
3. English
4. French
5. Italian

### Per language, we should collect 3 books, at minimum. Some of these should be close to each other in time or place of writing, as to avoid the model just learning to detect writing styles of specific authors.

### Now, as for the texts themselves: what rules should they meet?
1. The time and location of writing should be known or easily extrapolated.
2. The text shouldn't be modernized.
3. Most of the paragraphs of the book should be at least 500 characters.

### Because we are going to split the texts to train on their paragraphs, the third rule is to filter out short results. As per reference, the following excerpt from Locke's 'An Essay Concerning Humane Understanding Vol II' is 521 characters long.
    "As the ideas men's words stand for are of different sorts, so the way of
    making known the ideas they stand for, when there is occasion, is also
    different. For though DEFINING be thought the proper way to make known
    the proper signification of words; yet there are some words that will
    not be defined, as there are others whose precise meaning cannot be made
    known but by definition: and perhaps a third, which partake somewhat of
    both the other, as we shall see in the names of simple ideas, modes, and
    substances."